# 2-Opt and 3-Opt CVRP

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import operator
from functools import reduce
import itertools

# Получаем список городов, на которых суммарный вес превышает вместимость грузовика
def subtourslice(tour, vehicle, max_vehicle):
    #print("--------------------------------Начало работы функции--------------------------------")
    #vehilce[i][0] - номер города, vehicle[i][1] - вес груза для перевозки
    capacity_used = np.zeros(len(vehicle))
    k = 0
    slice = []
    mass = [] # список всех весов грузов для перевозки
    for i in range(len(vehicle)):
        # capacity_used[i] - начальный вес
        while((capacity_used[i] <= max_vehicle) and (vehicle[k][0] <= (len(tour) - 3))):
            capacity_used[i] += vehicle[k][1] # заполняем грузовик конкретным кол-вом груза для каждого города
            if(capacity_used[i] > max_vehicle):
                # если кол-во груза, который нужно отвезти, больше чем вместимость грузовика, то вычитаем это кол-во груза
                capacity_used[i] -= vehicle[k][1]
                slice.append(vehicle[k-1][0]) # запоминаем номер тура, чтобы повторно не проходить по такому же маршруту
                break
            k += 1
        mass.append(capacity_used[i])
    slice.append(vehicle[k-1][0])
    return(slice, mass)

def subtour(slice, tour):
    sub = []
    # добавляем номера городов, после которых суммарный вес груза превышал объем грузовика
    sub.append(tour[:(slice[0] + 1)])
    for i in range(0, len(slice) - 1):
        # нарежем маршруты городов до момента, пока суммарный вес груза не превысел объем грузовика
        sub.append(tour[(slice[i] + 1):(slice[i + 1] + 1)])
    return sub

def total_distance(lst, distance):
    d = 0
    if(len(lst) == 1):
        d = d + distance[(0, lst[0])]
        return d

    for n in range(1, len(lst)-1):
        i = lst[n]
        j = lst[n+1]
        d = d + distance[(i, j)]
    
    d = d + distance[(0, len(lst) - 1)]
    
    if(lst[0] == 0):
        d = d + distance[(0, lst[1])]
    else:
        d = d + distance[(0, lst[0])]
    return d

def Nearest_Neighbour(starting_node, cities, distance, q, max_vehicle):
    #Поставим стартовый узел и добавим его в список узлов
    NN = [starting_node]
    n = len(cities)
    
    while(len(NN) < n):
        k = NN[-1] # берём последний узел
        # считаем от этого узла расстояние до всевозможных остальных узлов и берем минимальное
        nn_len = []
        nn_idx = []
        for j in cities:
            if((k!=j) and (j not in NN)):# and (q[k][1] + q[j][1] < max_vehicle)):
                nn_idx.append((k, j))
                nn_len.append(distance[(k, j)])
#         nn = {(k,j): distance[(k, j)] for j in cities if((k!=j) and (j not in NN))}
        min_idx = nn_idx[0][1]
        min_len = nn_len[0]
        for i in range(0, len(nn_len)-1):
            if(nn_len[i] < min_len):
                if(q[nn_idx[i][0]][1] + q[nn_idx[i][1]][1] < max_vehicle):
                    min_len = nn_len[i]
                    min_idx = nn_idx[i][1]
#         new = min(nn.items(), key = lambda x: x[1]) # находим минимальное расстояние до узла
        NN.append(min_idx) # запоминае новый узел
    
    NN.append(starting_node)
    print("NN", NN)
    lst_NN, _ = subtourslice(NN, q, max_vehicle)
    print("lst_NN", lst_NN)
    all_lst_NN = subtour(lst_NN, NN)
    return all_lst_NN

def Two_Opt(NN, distance):
    for k in range(len(NN)):
        min_change = 0
        change = 0
        min_i = 0
        min_j = 0
        for i in range(len(NN[k]) - 2):
            for j in range(i+2, len(NN[k])-1):
                cost_actual = distance[(NN[k][i], NN[k][i+1])] + distance[(NN[k][j], NN[k][j+1])]
                #Убираем 2 ребра и вставляем 2 новых
                cost_new = distance[(NN[k][i], NN[k][j])] + distance[(NN[k][i+1], NN[k][j+1])]
                #Далее проверяем улучшился маршрут после перестановки или нет
                change = cost_new - cost_actual
                if(change < min_change):
                    min_change = change
                    #Также запомни индексы, на которых при перестановки маршрут улучшился
                    min_i = i
                    min_j = j

        if(min_change < 0):
             NN[k][min_i+1:min_j+1] = NN[k][min_i+1:min_j+1][::-1] # переворачиваем целый список городов
    return NN

from enum import Enum


class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"

def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        res = graph[(A, B)] + graph[(E, F)] - (graph[(B, F)] + graph[(A, E)])
        return res
    elif case == OptCase.opt_case_3:
        # ABC'
        res = graph[(C, D)] + graph[(E, F)] - (graph[(D, F)] + graph[(C, E)])
        return res
    elif case == OptCase.opt_case_4:
        # A'BC'
        res = graph[(A, B)] + graph[(C, D)] + graph[(E, F)] - (graph[(A, D)] + graph[(B, F)] + graph[(E, C)])
        return res
    elif case == OptCase.opt_case_5:
        # A'B'C
        res = graph[(A, B)] + graph[(C, D)] + graph[(E, F)] - (graph[(C, F)] + graph[(B, D)] + graph[(E, A)])
        return res
    elif case == OptCase.opt_case_6:
        # AB'C
        res = graph[(B, A)] + graph[(D, C)] - (graph[(C, A)] + graph[(B, D)])
        return res
    elif case == OptCase.opt_case_7:
        # AB'C'
        res = graph[(A, B)] + graph[(C, D)] + graph[(E, F)] - (graph[(B, E)] + graph[(D, F)] + graph[(C, A)])
        return res
    elif case == OptCase.opt_case_8:
        # A'B'C
        res = graph[(A, B)] + graph[(C, D)] + graph[(E, F)] - (graph[(A, D)] + graph[(C, F)] + graph[(B, E)])
        return res

def all_segments(n):
    """Generate all segments combinations"""
    return ((i, j, k)
        for i in range(n)
        for j in range(i + 2, n - 1)
        for k in range(j + 2, n - 1 + (i > 0)))

def reverse_segments(route, case, i, j, k, distance):
    """
    Create a new tour from the existing tour
    Args:
        route: existing tour
        case: which case of opt swaps should be used
        i:
        j:
        k:
    Returns:
        new route
    """
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
#     k = list(first_segment) + list(second_segment) + list(third_segment)
#     if(total_distance(k, distance) < total_distance(solution, distance)):
#         solution = first_segment + second_segment + third_segment
    return solution

def three_opt(tour, distance):
    """Iterative improvement based on 3 exchange."""
    new_tour = []
    for p in range(len(tour)):
        old_sub_tour = tour[p]
        moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                      OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                      OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0}
        improved = True
        best_found_route = tour[p]
#         if(len(best_found_route) > 5):
        while improved:
            improved = False
            for (i, j, k) in all_segments(len(tour[p])):
                # we check all the possible moves and save the result into the dict
                for opt_case in OptCase:
                    moves_cost[opt_case] = get_solution_cost_change(distance, best_found_route, opt_case, i, j, k)
                # we need the minimum value of substraction of old route - new route
                best_return = max(moves_cost, key=moves_cost.get)
#                 print("Movecost:", moves_cost[best_return])
                if moves_cost[best_return] > 0:
                    best_found_route = reverse_segments(best_found_route, best_return, i, j, k, distance)
                    improved = True
                    break
        print("OLD", total_distance(old_sub_tour, distance))
        print("NEW", total_distance(best_found_route, distance))
        if(total_distance(old_sub_tour, distance) < total_distance(best_found_route, distance)):
            new_tour.append(old_sub_tour)
        else:
            new_tour.append(best_found_route)
    return new_tour

In [4]:
n = 501
cities = [i for i in range(n)] # список номеров городов

roads = [(i, j) for i in range(n) for j in range(n) if(i != j)] # это попарные маршруты между всеми городами

# np.random.seed(0)
x = np.random.rand(n) * 10
y = np.random.rand(n) * 10


distance = {(i,j):np.hypot(x[i] - x[j], y[i]-y[j]) for i,j in roads}

N = [i for i in range(1, n)] # tour без депо

min_capasity = 10
max_capasity = 420
max_vehicle = 700
capacity = np.minimum(np.maximum(np.abs(np.random.normal(15, 10, size=[1, n])), min_capasity), max_capasity)

p = 1
max_vehicle *= p
while(max_vehicle < max_capasity):
    max_vehicle = max_vehicle / p
    p += 1
    max_vehicle *= p
print("Используется грузовиков: ", p)

q = []
for i in range(1, len(N)+1):
    q.append((i, capacity[0][i])) # вес груза, который необходимо перевести в город

time_start = time.time()

starting_node = 0
NN = Nearest_Neighbour(starting_node, cities, distance, q, max_vehicle) # Создаем начальный тур, который уже каким-то образом оптимизирован
solution1 = NN.copy()
print("Start solution", NN)

len_start_solution = best_solution = reduce(operator.add, (total_distance(x, distance) for x in solution1), 0) 

# while(best_solution >= len_start_solution):
flag = 1 
k1 = 0 # кол-во шагов
print("Start distance", reduce(operator.add, (total_distance(x, distance) for x in solution1), 0))
while(flag != 0):
    k1 += 1
    d_start1 = reduce(operator.add, (total_distance(p, distance) for p in solution1), 0)
    #print("start", d_start)
    solution1 = Two_Opt(solution1, distance).copy()
    #print("solution_two_opt", solution1)
    final1 = reduce(operator.add, (total_distance(p, distance) for p in solution1), 0)
    flag = np.abs(final1 - d_start1)
    if(best_solution > final1):
        best_solution = final1

flag = 1 
k2 = 0 # кол-во шагов
solution2 = NN.copy()
best_solution_1 = reduce(operator.add, (total_distance(x, distance) for x in solution2), 0) 
while(flag > 0):
    k2 += 1
    d_start2 = reduce(operator.add, (total_distance(p, distance) for p in solution2), 0)
    #print("start", d_start)
    solution2 = three_opt(solution2, distance).copy()
    #print("solution_three_opt", solution2)
    final2 = reduce(operator.add, (total_distance(p, distance) for p in solution2), 0)
    flag = np.abs(final2 - d_start2)
    print(final2)
time_final = time.time()

print(" ")
print("Result Two-Opt")
print("Optimize solution", solution1)
print("Optimize distance", final1)
print("Best Optimize distance", best_solution)
print("Steps", k1)
print(" ")
print("Result Three-Opt")
print("Optimize solution", solution2)
print("Optimize distance", final2)
print("Work time", time_final - time_start)
print("Steps", k2)

Используется грузовиков:  1
NN [0, 351, 240, 466, 374, 220, 399, 40, 167, 234, 138, 36, 74, 276, 495, 381, 225, 90, 142, 271, 256, 490, 64, 189, 236, 170, 332, 338, 209, 65, 314, 32, 223, 283, 95, 173, 49, 43, 277, 199, 136, 9, 144, 103, 114, 12, 96, 196, 238, 426, 489, 397, 172, 68, 400, 353, 263, 109, 127, 81, 26, 388, 119, 212, 447, 253, 46, 177, 337, 184, 61, 251, 265, 358, 112, 57, 137, 370, 88, 31, 452, 492, 303, 270, 39, 169, 482, 150, 486, 149, 226, 140, 290, 181, 382, 29, 42, 156, 52, 345, 38, 403, 55, 347, 8, 487, 175, 454, 183, 418, 118, 295, 323, 161, 356, 73, 244, 386, 413, 258, 441, 41, 176, 215, 213, 120, 92, 391, 325, 339, 228, 219, 393, 201, 76, 281, 34, 427, 484, 166, 174, 241, 105, 331, 141, 296, 352, 411, 107, 254, 47, 7, 366, 233, 448, 416, 406, 284, 210, 396, 86, 417, 286, 436, 35, 197, 497, 471, 115, 493, 478, 459, 171, 62, 126, 432, 301, 257, 97, 193, 230, 261, 311, 243, 54, 153, 287, 273, 394, 439, 237, 444, 104, 101, 429, 435, 204, 402, 165, 278, 498, 262, 151